# Data Ingestion

In [17]:
# !pip install -U langchain langchain-community langchain-openai
# !pip uninstall -y langchain langchain-core langchain-community langchainhub langchain-openai



In [18]:
## document structure


from langchain_core.documents import Document 

In [19]:
doc=Document(
    page_content="This is the content of the document.",
    metadata={"source": "generated", "author": "AI"},
)

In [20]:
doc

Document(metadata={'source': 'generated', 'author': 'AI'}, page_content='This is the content of the document.')

In [21]:
## creating a simple txt document

import os
os.makedirs("../data/text_files", exist_ok=True)

In [22]:
sample_text = {
    
    "../data/text_files/doc1.txt": """This is the content of document , it is about python programming 
and its applications in data science.
Python is a versatile language that is widely used in various fields.
keyfeatures include simplicity, readability, and a vast ecosystem of libraries.
  
it has verstaile uses including web development, data analysis, machine learning, and automation.
  
    """,

}


for file_path, content in sample_text.items():   # using 2 variables, one for key and one for value
    with open(file_path, "w",encoding="utf-8") as f:
        f.write(content)
        
print("Sample text files created.")


Sample text files created.


In [23]:
## text loader

from langchain_community.document_loaders import TextLoader

loader=TextLoader("../data/text_files/doc1.txt", encoding="utf-8")  
document=loader.load()  # it is an object of the Textloader


In [24]:
document  # automatically have the metadata and page_content

[Document(metadata={'source': '../data/text_files/doc1.txt'}, page_content='This is the content of document , it is about python programming \nand its applications in data science.\nPython is a versatile language that is widely used in various fields.\nkeyfeatures include simplicity, readability, and a vast ecosystem of libraries.\n  \nit has verstaile uses including web development, data analysis, machine learning, and automation.\n  \n    ')]

In [25]:
## Directory Loader

from langchain_community.document_loaders import DirectoryLoader

# load all text files from directory 


dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt",  # pattern to match files
    loader_cls=TextLoader, ## loader class to use
    loader_kwargs={'encoding':'utf-8'},
    show_progress=False
)

documents=dir_loader.load()




In [26]:
documents

[Document(metadata={'source': '..\\data\\text_files\\doc1.txt'}, page_content='This is the content of document , it is about python programming \nand its applications in data science.\nPython is a versatile language that is widely used in various fields.\nkeyfeatures include simplicity, readability, and a vast ecosystem of libraries.\n  \nit has verstaile uses including web development, data analysis, machine learning, and automation.\n  \n    ')]

In [27]:
# !pip install pymupdf

In [28]:
## Directory Loader

from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

# load all text files from directory 


dir_loader=DirectoryLoader(
    "../data/pdf",
    glob="**/*.pdf",  # pattern to match files
    loader_cls=PyMuPDFLoader, ## loader class to use,better for complex pdfs
    show_progress=False
)

pdf_docs = dir_loader.load()

pdf_docs


"""we are not getting one document per PDF file.
we are getting one document per page."""

'we are not getting one document per PDF file.\nwe are getting one document per page.'

In [29]:
type(pdf_docs[0])

langchain_core.documents.base.Document

## pdf loader to embeddings to vector database

In [30]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [31]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 4 PDF files to process

Processing: bert.pdf
Loaded 16 pages

Processing: gpt_4_technicalreport.pdf
Loaded 100 pages

Processing: rag.pdf
Loaded 19 pages

Processing: sample_test.pdf
Loaded 100 pages

Total documents loaded: 235


In [32]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [33]:
chunks=split_documents(all_pdf_documents)
chunks

Split 235 documents into 951 chunks

Example chunk:
Content: BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding
Jacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova
Google AI Language
{jacobdevlin,mingweichang,kentonl,kristout...
Metadata: {'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-05-28T00:07:51+00:00', 'author': '', 'keywords': '', 'moddate': '2019-05-28T00:07:51+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\bert.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1', 'source_file': 'bert.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-05-28T00:07:51+00:00', 'author': '', 'keywords': '', 'moddate': '2019-05-28T00:07:51+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\bert.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1', 'source_file': 'bert.pdf', 'file_type': 'pdf'}, page_content='BERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\nGoogle AI Language\n{jacobdevlin,mingweichang,kentonl,kristout}@google.com\nAbstract\nWe introduce a new language representa-\ntion model called BERT, which stands for\nBidirectional Encoder Representations from\nTransformers. Unlike recent language repre-\nsentation models (Peters et al., 2018a; Rad-\nford et al., 2018), BERT is designed to pr

## embeddings and vectordb

In [35]:
import numpy as np 
from sentence_transformers import SentenceTransformer
import chromadb 

from chromadb.config import Settings 
import uuid
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity


In [36]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


## vector store

In [37]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 951


In [38]:
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-05-28T00:07:51+00:00', 'author': '', 'keywords': '', 'moddate': '2019-05-28T00:07:51+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\bert.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1', 'source_file': 'bert.pdf', 'file_type': 'pdf'}, page_content='BERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\nGoogle AI Language\n{jacobdevlin,mingweichang,kentonl,kristout}@google.com\nAbstract\nWe introduce a new language representa-\ntion model called BERT, which stands for\nBidirectional Encoder Representations from\nTransformers. Unlike recent language repre-\nsentation models (Peters et al., 2018a; Rad-\nford et al., 2018), BERT is designed to pr

In [39]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 951 texts...


Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Generated embeddings with shape: (951, 384)
Adding 951 documents to vector store...
Successfully added 951 documents to vector store
Total documents in collection: 1902


### Retriever Pipeline From VectorStore

In [40]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



In [41]:
rag_retriever